In [1]:
#importer les bibiliotheques
import requests
from bs4 import BeautifulSoup
import csv
import numpy as np
import pandas as pd

In [2]:
def getData(TypeVetements):
    """
    scraper les donnees pour chaque type de vetements feminins 
    et retourner un dataframe des donnees obtenues.
    Le meme code s'applique pour les vetements des hommes en 
    changeant la colonne du genre .
    """
    liste_names=[]
    liste_prix=[]
    liste_ranking=[]
    liste_promo=[]
    liste_oldprix=[]
    liste_livrGratuite=[]
    liste_brand=[]
    liste_category=[]
    liste_boutiqueoff=[]
    liste_ids=[]
    liste_price=[]
    liste_reviews=[]
    for i in range(1,40):
        req = requests.get('https://www.jumia.ma/'+TypeVetements+'/?page='+str(i)+'#catalog-listing')
        html_doc = req.text
        soup = BeautifulSoup(html_doc, 'html.parser')
        container=soup.find_all('article',class_='prd _fb col c-prd')
    
        for item in container:
            #scarper ids
            ids=item.find('a',class_='core')['data-id']
            liste_ids.append(ids)
            #scraper price$
            price=item.find('a',class_='core')['data-price']
            liste_price.append(price)
            #scraper categories
            category=item.find('a',class_='core')['data-category']
            liste_category.append(category)
            #scraper brand
            brand=item.find('a',class_='core')['data-brand']
            liste_brand.append(brand)
            #scraper les prix
            prix=item.find('div',class_='prc').text
            liste_prix.append(prix)
            #scraper old price
            
            old_prix=item.find('div',class_='old')
            
            if old_prix==None:
                    liste_oldprix.append(np.nan)
            else:
                    liste_oldprix.append(old_prix.text)
            #scraper promo
            promo=item.find('div',class_='tag _dsct _sm')
            if promo==None:
                    liste_promo.append(np.nan)
            else:
                liste_promo.append(promo.text)
            #scraper names
            name=item.find('a',class_='core')['data-name']
            liste_names.append(name)
            #scraper ranking
            ranking=item.find('div',class_='stars _s')
            if ranking==None:
                  liste_ranking.append(np.nan)
            else:
                     
                     liste_ranking.append(ranking.text)
            #scraper reviews
            reviews=item.find('div',class_='rev')
            if reviews==None:
                  liste_reviews.append(np.nan)
            else:
                     
                    liste_reviews.append(reviews.text.replace(ranking.text,''))
            #scraper livraison gratuite
            livrGratuite=item.find('p',class_='shipp')
            if livrGratuite==None:
                liste_livrGratuite.append('No')
            else:
                liste_livrGratuite.append('Yes')
            #scraper boutique officielle
            boutiqueoff=item.find('div',class_='tag _mall _sm')
            if boutiqueoff==None:
                liste_boutiqueoff.append('No')
            else:
                liste_boutiqueoff.append('Yes')
    liste_dates=['2022-03-25' for i in range(len(liste_ids))]
    liste_genre=['Femme' for i in range(len(liste_ids))]
    ar_femmes = np.array([pd.Series(liste_ids),pd.Series(liste_names),pd.Series(liste_brand),pd.Series(liste_category),pd.Series(liste_price),pd.Series(liste_boutiqueoff),pd.Series(liste_prix),pd.Series(liste_genre),pd.Series(liste_oldprix) , pd.Series(liste_promo) , pd.Series(liste_ranking),pd.Series(liste_reviews),pd.Series(liste_livrGratuite),pd.Series(liste_dates)]).T
    rows_femmes = ["id","name","brand","category","price$","boutique_officielle","price","gender","price_before","promotion","ranking","numberOfReviews","Livraison_gratuite","Date"]
    df_femmes = pd.DataFrame(ar_femmes,columns =rows_femmes)
    return df_femmes

In [3]:
#recuperer les dataframes pour quelques types de vetements
df_manteaux=getData('vestes-manteaux-blousons')
df_robes=getData('robes-femme')
df_hoodies=getData('vetements-mode-hoodies-sweat-shirts')
df_jeans=getData('jeans-femme')
df_combinaisons=getData('ensembles-combinaisons')
df_leggings=getData('leggings-femme')
df_lingerie=getData('lingerie-femme')
df_pantalons=getData('pantalons-femme')
df_shorts=getData('shorts')
df_jupes=getData('jupes')
df_chaussettes=getData('chaussettes-femmes')
df_costumes=getData('mode-femme-costumes-blazers')
df_pulls=getData('pulls-gilets')
df_maillots=getData('maillots-de-bain-femmes')
df_chemisiers=getData('tops-chemisiers')
df_kimono=getData('kimono')

In [4]:
#concatener tous les dataframes
df_vetements=pd.concat([df_chaussettes, df_chemisiers,df_combinaisons,df_costumes,df_hoodies,df_jeans,df_jupes,df_kimono,df_leggings,df_lingerie,df_maillots,df_manteaux,df_pantalons,df_pulls,df_robes,df_shorts])

In [31]:
#afficher les cinq premieres lignes
df_vetements.head()

,id,name,brand,category,price$,boutique_officielle,price,gender,price_before,promotion,reviews,numberofReviewers,Livraison_gratuite,Date
0,DE898MW12G9MONAFAMZ,Lot de 3 paires de chaussettes courtes à motif...,Defacto,Fashion/Women's Fashion/Clothing/Socks & Hosiery,4.58,Yes,49.00 Dhs,Femme,NaN,NaN,NaN,NaN,Yes,2022-03-25
1,GE232FA1KI4UONAFAMZ,Chaussettes Transparent Femmes 10 Paires,Generic,Fashion/Women's Fashion/Clothing/Socks & Hosiery,3.64,No,39.00 Dhs,Femme,99.00 Dhs,61%,3.8 out of 5,(70),Yes,2022-03-25
2,GE232MW13KZJ5NAFAMZ,Collant Femmes Transparent,Generic,Fashion/Women's Fashion/Clothing/Socks & Hosie...,3.64,No,39.00 Dhs,Femme,99.00 Dhs,61%,4.3 out of 5,(9),No,2022-03-25
3,GE232MW1FC6BENAFAMZ,Pack 12 Chaussettes Femmes Hommes 100 % Coto...,Generic,Fashion/Men's Fashion/Clothing/Socks,4.61,No,49.30 Dhs,Femme,99.00 Dhs,50%,3.5 out of 5,(329),No,2022-03-25
4,GE232MW0ASAU6NAFAMZ,"Chaussettes de pied anti-sèches, Silicone prot...",Generic,Fashion/Women's Fashion/Clothing/Socks & Hosie...,2.62,No,27.99 Dhs,Femme,NaN,NaN,3.7 out of 5,(9),No,2022-03-25


In [3]:
#recuperer le dataframe des vetements masculins
df_men_clothes=pd.read_csv('C:/jumia_men_clothes_.csv')

In [35]:
df=pd.concat([df_men_clothes, df_vetements])

,id,name,brand,category,price$,boutique_officielle,price,gender,price_before,promotion,reviews,numberofReviewers,Livraison_gratuite,Date
0,GE232FA1FQVNSNAFAMZ,Hoodie/Capuchon Death Note Anime - Otaku,Generic,Fashion/Men's Fashion/Clothing/Fashion Hoodies...,6.04,No,64.60 Dhs,Homme,233.00 Dhs,72%,3.2 out of 5,(81),No,2022-03-25
1,GE232MW1JMM00NAFAMZ,Hoodie Manga Naruto Anime Itachi Uchiha Sweats...,Generic,Fashion/Men's Fashion/Clothing/Fashion Hoodies...,13.92,No,149.00 Dhs,Homme,192.00 Dhs,22%,4 out of 5,(11),No,2022-03-25
2,GE232MW011137NAFAMZ,Hoodie Manga Anime Naruto Shippuden Sweatshirt...,Generic,Fashion/Men's Fashion/Clothing/Fashion Hoodies...,20.46,No,219.00 Dhs,Homme,260.00 Dhs,16%,4.5 out of 5,(19),No,2022-03-25
3,GE232FA1MGS7TNAFAMZ,Hoodie / Capuchon noir avec motif de ATTACK ON...,Generic,Fashion/Men's Fashion/Clothing/Fashion Hoodies...,7.9,No,84.55 Dhs,Homme,399.00 Dhs,79%,3.4 out of 5,(35),No,2022-03-25
4,GE232MW0X9UR9NAFAMZ,Hoodie Manga Anime Naruto Shippuden Sweatshirt...,Generic,Fashion/Men's Fashion/Clothing/Fashion Hoodies...,13.92,No,149.00 Dhs,Homme,285.00 Dhs,48%,3.3 out of 5,(33),No,2022-03-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
816,MC584MW1GNX1TNAFAMZ,Short en cote coton gris chiné,m&c,Fashion/Women's Fashion/Clothing/Shorts,3.74,Yes,40.00 Dhs,Femme,NaN,NaN,3 out of 5,(6),No,2022-03-25
817,MC584MW0AWWPTNAFAMZ,Cycliste femme Coton Blanc,m&c,Fashion/Women's Fashion/Clothing/Shorts,4.21,Yes,45.00 Dhs,Femme,NaN,NaN,2.6 out of 5,(10),No,2022-03-25
818,DE898MW08WM01NAFAMZ,SHORT - BLEU,Defacto,Fashion/Women's Fashion/Clothing/Shorts,36.44,Yes,390.00 Dhs,Femme,NaN,NaN,5 out of 5,(2),No,2022-03-25
819,KO207MW0JUXGPNAFAMZ,SHORT - BLANC,Koton,Fashion/Women's Fashion/Clothing/Shorts,25.14,Yes,269.00 Dhs,Femme,NaN,NaN,NaN,NaN,No,2022-03-25


In [36]:
df.to_csv ('Jumia_clothes.csv', index = False, header=True)